In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Input
from keras.callbacks import EarlyStopping

# Read data from the StudentsPerformace.csv

In [2]:
dataset = pd.read_csv("StudentsPerformance.csv")
dataset.head(5)

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [3]:
x = dataset.iloc[:,0:5]
y = dataset.iloc[:, 5:]

# One hot encode inputs

In [4]:
x = pd.get_dummies(x)
x.head(5)

,gender_female,gender_male,race/ethnicity_group A,race/ethnicity_group B,race/ethnicity_group C,race/ethnicity_group D,race/ethnicity_group E,parental level of education_associate's degree,parental level of education_bachelor's degree,parental level of education_high school,parental level of education_master's degree,parental level of education_some college,parental level of education_some high school,lunch_free/reduced,lunch_standard,test preparation course_completed,test preparation course_none
0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1
1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,1,1,0
2,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,1
3,0,1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1
4,0,1,0,0,1,0,0,0,0,0,0,1,0,0,1,0,1


# Split into training and testing datasets
## 70/15/15 split for Training/Test/Validation

In [5]:
# In the first step we will split the data in training and remaining dataset
X_train, X_remaining, y_train, y_remaining = train_test_split(x, y, test_size=0.3, random_state=10)

# Now since we want the valid and test size to be equal (15% each of overall data). 
# we have to define valid_size=0.5 (that is 50% of remaining data which is 30% of our original data size)
X_valid, X_test, y_valid, y_test = train_test_split(X_remaining, y_remaining, test_size=0.5, random_state=10)

# Create model

In [6]:
def create_model():
    model = Sequential()
    model.add(Input(shape=(17,)))
    model.add(Dense(30, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    return model
    
my_model = create_model()
my_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 30)                540       
                                                                 
 dense_1 (Dense)             (None, 3)                 93        
                                                                 
Total params: 633
Trainable params: 633
Non-trainable params: 0
_________________________________________________________________


# Compile and train the model using training and validation data

In [7]:
my_model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

# use EarlyStopping callback monitoring loss
# patience is the number of epochs with no improvement after which training will be stopped
callback = EarlyStopping(monitor='loss', patience=3)

In [8]:
# train model using training data and validation data
history = my_model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, callbacks=[callback], verbose=1, shuffle=True)

Epoch 1/50
22/22 [==============================] - 0s 10ms/step - loss: 4756.8882 - accuracy: 0.4300 - val_loss: 4810.0293 - val_accuracy: 0.5267
Epoch 2/50
22/22 [==============================] - 0s 1ms/step - loss: 4756.6240 - accuracy: 0.4843 - val_loss: 4809.8311 - val_accuracy: 0.4667
Epoch 3/50
22/22 [==============================] - 0s 1ms/step - loss: 4756.4009 - accuracy: 0.5000 - val_loss: 4809.6440 - val_accuracy: 0.4667
Epoch 4/50
22/22 [==============================] - 0s 2ms/step - loss: 4756.2080 - accuracy: 0.5257 - val_loss: 4809.4590 - val_accuracy: 0.5333
Epoch 5/50
22/22 [==============================] - 0s 1ms/step - loss: 4756.0273 - accuracy: 0.5686 - val_loss: 4809.2593 - val_accuracy: 0.5933
Epoch 6/50
22/22 [==============================] - 0s 2ms/step - loss: 4755.8467 - accuracy: 0.5857 - val_loss: 4809.0737 - val_accuracy: 0.6000
Epoch 7/50
22/22 [==============================] - 0s 1ms/step - loss: 4755.6924 - accuracy: 0.6043 - val_loss: 4808.9336 

## Evaluate the model using testing data

In [14]:
scores = my_model.evaluate(X_test, y_test, verbose=0)
print("Loss:", (round(scores[0], 2)))
print("Accuracy: %.2f%%" % (scores[1]*100))

Loss: 4813.07
Accuracy: 62.67%
